In [96]:
%matplotlib inline
import pandas as pd
import re
import matplotlib.pylab as plt
import emoji
from textblob import TextBlob
from mtranslate import translate
from datetime import datetime
import csv
from googletrans import Translator
import time

candidate = 'Arthur do Val_2020-11-14 23:39:18.049512'

#https://www.wordclouds.com/

In [97]:
tweets_df = pd.read_csv("tweets/"+candidate+".csv", header=None,  names=['createdAt', 'id', 'tweet'])

In [98]:
tweets_df.head()

,createdAt,id,tweet
0,Fri Nov 13 23:59:55 +0000 2020,1327400834657902593,@DaviLemes @Joaogrs__ Bom primeiramente vc n v...
1,Fri Nov 13 23:54:16 +0000 2020,1327399411874492421,@ManuelaDavila que isso to doido com tamanha e...
2,Fri Nov 13 23:49:23 +0000 2020,1327398185283821570,@obrunobla Covas: 40%\nBoulos: 18%\nFrança: 12...
3,Fri Nov 13 23:48:30 +0000 2020,1327397961068933122,"@lbertozzi Lendo rapidamente, achei que fosse ..."
4,Fri Nov 13 23:41:03 +0000 2020,1327396085082562562,@carteiroreaca @bernardokuster2 Arthur do val ...


In [99]:
stop_words = ["de","a","o","que","e","do","da", "das", "em","um","para","é","com","não","uma","os","no","se","na","por","mais","as","dos",
"como","mas","foi","ao","ele","das","tem","à","seu","sua","ou","ser","quando","muito","há","nos","já","está","eu","também","só","pelo",
"pela","até","isso","ela","entre","era","depois","sem","mesmo","aos","ter","seus","quem","nas","me","esse","eles","estão","você","tinha",
"foram","essa","num","nem","suas","meu","às","minha","têm","numa","pelos","elas","havia","seja","qual","será","nós","tenho","lhe","deles",
"essas","esses","pelas","este","fosse","dele","tu","te","vocês","vos","lhes","meus","minhas","teu","tua","teus","tuas","nosso","nossa",
"nossos","nossas","dela","delas","esta","estes","estas","aquele","aquela","aqueles","aquelas","isto","aquilo","estou","está","estamos",
"estão","estive","esteve","estivemos","estiveram","estava","estávamos","estavam","estivera","estivéramos","esteja","estejamos","estejam",
"estivesse","estivéssemos","estivessem","estiver","estivermos","estiverem","hei","há","havemos","hão","houve","houvemos","houveram",
"houvera","houvéramos","haja","hajamos","hajam","houvesse","houvéssemos", "haveria", "houvessem","houver","houvermos","houverem","houverei",
"houverá", "houveremos","houverão","houveria","houveríamos","houveriam","sou","somos","são","era","éramos","eram","fui","foi","fomos","foram","fora",
"fôramos","seja","sejamos","sejam","fosse","fôssemos","fossem","for","formos","forem","serei","será","seremos","serão","seria","seríamos",
"seriam","tenho","tem","temos","tém","tinha","tínhamos","tinham","tive","teve","tivemos","tiveram","tivera","tivéramos","tenha","tenhamos",
"tenham","tivesse","tivéssemos","tivessem","tiver","tivermos","tiverem","terei","terá","teremos","terão","teria","teríamos","teriam", "vai",
"vou", "tão", "alguma", "interesse", "ter", "caso", "abaixo", "animais", "ainda", "outras", "etc.", "em", "a", "b", "c", 
"d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "x", "y", "z", ".", ",", "-", "#", "pra", 
"tá", "tudo", "agora", "dia", "brasil", "nada", "nessa", "ano", 
"comprei", "aqui", "r$", "reais", "tava", "gente", "ver", "produtos", "todo", "produto", "antes", "loja", "pq", "aí", "fazer",
"ta", "semana", "vc", "coisa", "1", "brasileiro", "povo", "?", "to", "queria", "2", "ontem", "compra", "tô", "vi",
"sempre", "sendo", "quero", "bem", "coisas", "dias", "bom", "nunca", "vcs", "pessoas", "mês", "pode", "3", "pro", "faz", "olha",
"nao", "menos", "friday", "lá", "assim", "real", "melhor", "sabe", "realmente", "!", "fazendo", "vamos", "hj", "dessa", 
"boa", "ai", "fica", "dar", "quer", "porque", "sei", "vendo", "alguém", "dá", "sobre", "onde", "verdade", "única", "começou", "dizer", 
"apenas", "ficar", "uns", "vão", "acho", "kkkk", "vez", "desde", "kkk", "algo", "normal", "dois", "atrás", "mundo", "chega", 
"pois", "10", "desse", "sim", "...", "6", "galera", "nesse", "duas", "né", "alguns", "algumas", "ia", "fazem", '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'etc',
"r$", ",", ".", "-", "'", "@", "#", "$", "%", "&", "*", "(", ")", "_", "+", "<", ">", "{", "}", "[", "]", "\\", "/", "\n",
"\t", "...", ":", ";", "!", "?", "", "ir", "ja", "ué", "tal", "fiz", "né", "é", "ei", "ñ", "fez", "sim", "!!!", "todos", '"', '|', '2020', 'sp', 'São Paulo', 'Sao Paulo']

In [100]:
def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    
    return clean_text
 
def clean_tweet(tweet):

        #text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split())

        #text = text.replace('.', '')
        '''
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

        text = emoji_pattern.sub(r'', text)
        '''

        tweet = give_emoji_free_text(tweet)
        

        return tweet

def process_text(texts):
    processed_tweets = []
    
    for text in texts:
        text = str(text)
        text = text.lower()

        text = re.sub('http://\S+|https://\S+', '', text)
        text = re.sub('http[s]?://\S+', '', text)
        
        text = clean_tweet(text)
            
        processed_tweets.append(text)
        
    return processed_tweets

In [101]:
def get_texts_by_word(texts, words):
    tweets_with_word = []
    
    for text in texts:
        for word in words:
            if(word in text):
                tweets_with_word.append(text)
                continue
        
    return tweets_with_word

In [102]:
def count_word_in_tweets(texts, terms, remove_arroba):
    dict_words = {}
    
    for text in texts:
        words = text.split(' ')

        for word in words:
            word = word.replace('!', '')
            word = word.replace('?', '')
            word = word.replace('.', '')
            word_is_ok = True 
            if word not in stop_words:
                for term in terms:
                    if term in word:
                        word_is_ok = False
                if (remove_arroba and ('@' in word or '#' in word)):
                    word_is_ok = False
                if(word_is_ok and word in dict_words):
                    count = dict_words[word]
                    count += 1
                    dict_words[word] = count 
                else:
                    dict_words[word] = 1
            
    dictWordsSorted = sorted(dict_words.items(), key=lambda x:x[1], reverse=True)
    
    return dictWordsSorted

In [103]:
def count_word(texts):
    dict_words = {}
    
    for text in texts:
        words = text.split(' ')

        for word in words:
            word = word.replace('!', '')
            word = word.replace('?', '')
            word = word.replace('.', '')
            word_is_ok = True 
            if word not in stop_words:
                if(word in dict_words):
                    count = dict_words[word]
                    count += 1
                    dict_words[word] = count 
                else:
                    dict_words[word] = 1
            
    dictWordsSorted = sorted(dict_words.items(), key=lambda x:x[1], reverse=True)
    
    return dictWordsSorted

In [104]:
processed_text = process_text(tweets_df['tweet'].tolist())

In [105]:
tweets_df['processed_text'] = processed_text

In [106]:
tweets_df.head()

,createdAt,id,tweet,processed_text
0,Fri Nov 13 23:59:55 +0000 2020,1327400834657902593,@DaviLemes @Joaogrs__ Bom primeiramente vc n v...,@davilemes @joaogrs__ bom primeiramente vc n v...
1,Fri Nov 13 23:54:16 +0000 2020,1327399411874492421,@ManuelaDavila que isso to doido com tamanha e...,@manueladavila que isso to doido com tamanha e...
2,Fri Nov 13 23:49:23 +0000 2020,1327398185283821570,@obrunobla Covas: 40%\nBoulos: 18%\nFrança: 12...,@obrunobla covas: 40% boulos: 18% frança: 12% ...
3,Fri Nov 13 23:48:30 +0000 2020,1327397961068933122,"@lbertozzi Lendo rapidamente, achei que fosse ...","@lbertozzi lendo rapidamente, achei que fosse ..."
4,Fri Nov 13 23:41:03 +0000 2020,1327396085082562562,@carteiroreaca @bernardokuster2 Arthur do val ...,@carteiroreaca @bernardokuster2 arthur do val ...


In [107]:
def created_text_count_word(words):
    text = ''
    for word in words:
        for index in range(0, word[1]):
            text = text + ' ' + word[0]
            
        print(str(word[1]) + ' ' + word[0])
    return text

In [108]:
tweets_text = get_texts_by_word(tweets_df['processed_text'].tolist(), ['boulos'])
count_dict_tweets = count_word_in_tweets(tweets_text, ['boulos'], False)
count_tweets_words = created_text_count_word(count_dict_tweets)

213 arthur
186 val
36 #debatenacultura
31 covas
27 tatto
27 –
24 guilherme
23 debate
22 bruno
21 frança
18 russomanno
18 val,
18 russomano
17 covas,
17 jilmar
15 joice
15 paulo
15 mamãe
13 márcio
13 #debatecultura
12 turno
11 cara
11 pesquisas
11 candidato
10 51
10 fake
10 fala
9 celso
9 (psol)
9 russomano,
9 paraná
8 único
8 prefeito
8 15,2%
8 (+1,8)
8 falei
8 tatto,
8 news
8 disse
8 vídeo
7 2º
7 contra
7 15%
7 @arthurmoledoval
7 segundo
7 votar
7 eleitoral
7 #vaidar51
7 proposta
7 30,8%
7 12,9%
7 11,8%
7 5,2%
7 (+1)
7 matarazzo
7 (+2)
7 acabar
7 falei,
7 gato
7 frança,
7 pergunta
6 propostas
6 vote
6 marina
6 prefeitura
6 (pt)
6 #domingoe51
6 (psdb)
6 (psb)
6 3,5%
6 cu
6 deve
6 resposta
6 fogo
6 falou
6 prédio
6 cof
5 5%
5 outros:
5 6%
5 (psdb):
5 (psol):
5 10%
5 (psb):
5 (patriota):
5 (pt):
5 helou
5 votos
5 válidos
5 #eleições2020
5 sério
5 falando
5 pior
5 leva
5 2°
5 2,4%
5 hasselmann
5 1,8%
5 parece
5 popular
5 valem
5 enterrou
5 #nãodonadorianãodonachina
5 #fechadocomlevyfideli

In [109]:
def get_words_more_used_by_term(tweets, term, term2=''):
    if(term2 == ''):
        term2 = term
    dict_words = {}
    tweets_by_term = []
    for tweet in tweets:
        if(term in tweet or term2 in tweet):
            tweets_by_term.append(tweet)
        
    for text in tweets_by_term:
        words = text.split(' ')

        for word in words:
            if word not in stop_words and term not in word and term2 not in word:
                if(word in dict_words):
                    count = dict_words[word]
                    count += 1
                    dict_words[word] = count 
                else:
                    dict_words[word] = 1
            
    dictWordsSorted = sorted(dict_words.items(), key=lambda x:x[1], reverse=True)
        
    
    return tweets_by_term, dictWordsSorted

In [110]:
tweets_by_term, dictWordsSorted = get_words_more_used_by_term(tweets_df['processed_text'].tolist(), 'boulos', '')

In [111]:
dictWordsSorted

[('arthur', 212),
 ('val', 175),
 ('#debatenacultura', 36),
 ('covas', 29),
 ('–', 27),
 ('tatto', 25),
 ('guilherme', 24),
 ('bruno', 22),
 ('frança', 21),
 ('debate', 21),
 ('val,', 18),
 ('russomano', 18),
 ('covas,', 17),
 ('russomanno', 17),
 ('jilmar', 17),
 ('joice', 15),
 ('mamãe', 15),
 ('paulo', 14),
 ('márcio', 13),
 ('#debatecultura', 13),
 ('cara', 11),
 ('pesquisas', 11),
 ('candidato', 10),
 ('fake', 10),
 ('fala', 10),
 ('celso', 9),
 ('russomano,', 9),
 ('51', 9),
 ('paraná', 9),
 ('único', 8),
 ('(psol)', 8),
 ('prefeito', 8),
 ('val.', 8),
 ('15,2%', 8),
 ('(+1,8)', 8),
 ('tatto,', 8),
 ('disse', 8),
 ('2º', 7),
 ('contra', 7),
 ('15%', 7),
 ('@arthurmoledoval', 7),
 ('segundo', 7),
 ('turno', 7),
 ('#vaidar51', 7),
 ('30,8%', 7),
 ('12,9%', 7),
 ('11,8%', 7),
 ('5,2%', 7),
 ('(+1)', 7),
 ('(+2)', 7),
 ('acabar', 7),
 ('falei', 7),
 ('falei,', 7),
 ('gato', 7),
 ('frança,', 7),
 ('pergunta', 7),
 ('vote', 6),
 ('marina', 6),
 ('prefeitura', 6),
 ('(pt)', 6),
 ('votar

In [112]:
def getWordNegatives(self):

        for tweet in tweetsNegatives:
            words = tweet.split()

            for word in words:
                if(word not in stop_words):
                    existe = False
                    if(word in dictNegatives):
                        existe = True

                    if(not existe):
                        dictNegatives[word] = 1
                    if(existe):
                        count = dictNegatives[word]
                        count += 1
                        dictNegatives[word] = count

def getWordPositives(self):

    for tweet in tweetsPositives:
        words = tweet.split()

        for word in words:
            word = word.lower()
            if(word not in stop_words):
                existe = False
                if(word in dictPositive):
                    existe = True

                if(not existe):
                    dictPositive[word] = 1
                if(existe):
                    count = dictPositive[word]
                    count += 1
                    dictPositive[word] = count

In [113]:
tweetsPositives = []
tweetsNegatives = []
dictPositive = {}
dictNegatives = {}

tweets = []
tweetsText = []
dictWords = {}

In [120]:
'''
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.com.br',
      'translate.google.co.kr',
      'translate.google.com.vn',
      'translate.google.com.ua',
      'translate.google.com.hk',
      'translate.google.com.tr',
      'translate.google.co.in',
      'translate.google.co.jp',
      'translate.google.co.uk',
      'translate.google.cn',
      'translate.google.de',
      'translate.google.fr',
      'translate.google.it',
      'translate.google.pl',
      'translate.google.ru',
    ])
'''

translator = Translator()

def get_tweet_sentiment(tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        translated = None
        print(tweet)
        #tweet = tweet
        try:
            print(tweet)
            translated = translator.translate(tweet, src='pt', dest='en').text
            print("TRANSLATED:", translated)
            print('\n')
        except Exception as e:
            print("Error: TRANSLATION ERROR OR TRANSLATION EXCEPTION")
            print(translated)
            print(e)
            return -1, -1

        try:
            translatedTextBlob = TextBlob(translated)
        except Exception as e:
            print("Error: Error trying to use TextBlob API")
            print(translated)
            print(e)
            return -1, -1

        # set sentiment
        if translatedTextBlob.sentiment.polarity > 0:
            tweetsPositives.append(tweet)
            return translated, 'positive'
        elif translatedTextBlob.sentiment.polarity == 0:
            return translated, 'neutral'
        else:
            tweetsNegatives.append(tweet)
            return translated, 'negative'

In [122]:
parsed_tweet = {}
tweets = []
tweetG = None
try:
    now = datetime.now()
    csvFile = open(candidate+'_translated_sentiment.csv', 'w', encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    
    for tweet, id in zip(tweets_df['processed_text'].values, tweets_df['id'].values):
        parsed_tweet['text'] = tweet
        tweetG = tweet
        #print(tweet)
        #print(get_tweet_sentiment(tweet))
        
        translated, sentiment = get_tweet_sentiment(tweet)
        if(translated != -1):
            parsed_tweet['id'] = id
            parsed_tweet['sentiment'] = sentiment
            parsed_tweet['translated'] = translated
            tweets.append(parsed_tweet)
            csvWriter.writerow([id, tweet, sentiment, translated])
        parsed_tweet = {}
        #time.sleep(2) # this will prevent google translate from starting to refuse requests
        #break
except Exception as e:
        print("\n\n\n")
        print(get_tweet_sentiment(tweetG))
        print("ERROR", e)

@davilemes @joaogrs__ bom primeiramente vc n viu as propostas do arthur do val que é o mais preparado e com as melhores propostas se quiser ver quais são pra pelo menos saber quem está criticando veja o link abaixo
@davilemes @joaogrs__ bom primeiramente vc n viu as propostas do arthur do val que é o mais preparado e com as melhores propostas se quiser ver quais são pra pelo menos saber quem está criticando veja o link abaixo
TRANSLATED: @davilemes @joaogrs__ good first you didn't see arthur do val's proposals which is the most prepared and with the best proposals if you want to see which ones are at least to know who is criticizing see the link below


@manueladavila que isso to doido com tamanha escrotice do arthur do val não dá para elogiar nem uma vez ele caga com tudo
@manueladavila que isso to doido com tamanha escrotice do arthur do val não dá para elogiar nem uma vez ele caga com tudo
TRANSLATED: @manueladavila this is crazy with so much scrutiny by arthur do val you can’t prai

TRANSLATED: @carteiroreaca @ bernardokuster2 I will follow the advice of bolsonaro and vote for someone who does not use fundão! arthur do val 51 # vaidar51


@arthurmoledoval arthur "nanico" do val está no nível do fidelyx e o cabo daciolo. só que ninguém quer socar os excandidatos a presidência.
@arthurmoledoval arthur "nanico" do val está no nível do fidelyx e o cabo daciolo. só que ninguém quer socar os excandidatos a presidência.
TRANSLATED: @arthurmoledoval arthur "nanico" do val is at the level of fidelyx and cabo daciolo. but nobody wants to punch the ex-presidents.


@celsorussomanno #domingoé51 são paulo tem jeito, basta a população querer, arthur do val prefeito, conheça o arthur, de uma chance a sp
@celsorussomanno #domingoé51 são paulo tem jeito, basta a população querer, arthur do val prefeito, conheça o arthur, de uma chance a sp
TRANSLATED: @celsorussomanno # domingoé51 são paulo has a way, just the population wants, arthur do val mayor, meet arthur, from a chance to sp

TRANSLATED: @obrunobla guess for valid votes in the city of são paulo: 1st - caves (psdb): 38% 2nd - boulos (psol): 15% 3rd - russomanno (prb): 14% 4th - france (psb): 10% 5th - arthur do val (patriot): 8% 6th - tatto (en): 6% others: 9% # elections2020


mano, to lendo as propostas dos prefeitos, e o arthur do val tem apenas dois parágrafos de assistência social, mano, eu nas minhas redações escrevia mais que isso. tirando os erros de ortografia, ele n específica nada. quem votar nele tem algum tipo de problema pqp
mano, to lendo as propostas dos prefeitos, e o arthur do val tem apenas dois parágrafos de assistência social, mano, eu nas minhas redações escrevia mais que isso. tirando os erros de ortografia, ele n específica nada. quem votar nele tem algum tipo de problema pqp
TRANSLATED: man, reading the mayors' proposals, and arthur do val has only two paragraphs of social assistance, man, I wrote more than that in my newsrooms. apart from spelling errors, it doesn’t specify anything

TRANSLATED: @allanldsantos yesterday i was going towards usp and i passed a bridge and it had a banner on it: if it supports arthur do val, honk.


@julianajmarcal @willian36399107 @gabs_kysdr @felipeneto a atitude não foi do arthur do val e esse momento da live foi tirado de contexto
@julianajmarcal @willian36399107 @gabs_kysdr @felipeneto a atitude não foi do arthur do val e esse momento da live foi tirado de contexto
TRANSLATED: @julianajmarcal @ willian36399107 @gabs_kysdr @felipeneto the attitude was not arthur do val and this moment of the live was taken out of context


arthur do val vem ganhando grande visibilidade no cenário político brasileiro desde 2017, sendo hoje candidato à prefeitura do município. conheça suas propostas pesquisa completa: #eleicoes2020 #arthurdoval #mamaefalei
arthur do val vem ganhando grande visibilidade no cenário político brasileiro desde 2017, sendo hoje candidato à prefeitura do município. conheça suas propostas pesquisa completa: #eleicoes2020 #ar

TRANSLATED: @caioblumerr @soufadosfrs arthur do val, smell my dick! mama I said!


@rogeriomorgado @arthurmoledoval votando no arthur do val vc manda boulos para segundo turno , parabéns pela burrice , tem que votar 10
@rogeriomorgado @arthurmoledoval votando no arthur do val vc manda boulos para segundo turno , parabéns pela burrice , tem que votar 10
TRANSLATED: @rogeriomorgado @arthurmoledoval voting for arthur do val u send boulos for second round, congratulations on stupidity, you have to vote 10


será que a galerinha do tt vai mostrar a fake news latente do celso russomanno, que acusou o arthur do val (#vaidar51) de faltar na alesp, sendo que o próprio falta na câmara dos deputados? 97% de presença e isso pode ser conferido no site oficial da alesp. #arthurdoval51
será que a galerinha do tt vai mostrar a fake news latente do celso russomanno, que acusou o arthur do val (#vaidar51) de faltar na alesp, sendo que o próprio falta na câmara dos deputados? 97% de presença e isso pode 

...

TRANSLATED: @henriolliveira_ let's go for arthur do val in the second round # vaidar51


@madruga005 russo mano que apoiou dilma é tem nome na planilha da odebrecht ? não caiam nessa vamos por o arthur do val no segundo turno #vaidar51
@madruga005 russo mano que apoiou dilma é tem nome na planilha da odebrecht ? não caiam nessa vamos por o arthur do val no segundo turno #vaidar51
TRANSLATED: @ madruga005 russian homie who supported dilma's name on odebrecht's spreadsheet? don't fall for this let's put arthur do val in the second round # vaidar51


bruno covas nunca foi de direita, é um político tradicional de centro que distribui secretarias até pra gente do psol. russumanno é um impostor oportunista que já apoiou lula, dilma e agora se agarra ao bolsonaro. o único candidato de direita é arthur do val. #domingoé51
bruno covas nunca foi de direita, é um político tradicional de centro que distribui secretarias até pra gente do psol. russumanno é um impostor oportunista que já apoiou lula

TRANSLATED: @cesarfilho arthur do val 51 the only one who gave up spending people's money on campaign. with respect to taxpayer 51 in them. @arthurmoledoval


'covas é um dos piores prefeitos de sp', diz arthur do val em sua chegada ao debate na tv cultura
'covas é um dos piores prefeitos de sp', diz arthur do val em sua chegada ao debate na tv cultura
TRANSLATED: 'covas is one of the worst mayors of sp', says arthur do val on his arrival at the debate on tv cultura


#naovotenopsdb #vaidar51 #arrancada51 vai dar 51 arthur do val mamaefalei.
#naovotenopsdb #vaidar51 #arrancada51 vai dar 51 arthur do val mamaefalei.
TRANSLATED: #naovotenopsdb # vaidar51 # arrancada51 will give 51 arthur do val mamaefalei.


@roxmo @lzmrtn66 @reclameaqui @loggi vote mamãe falei para prefeito de são paulo 51 #vaidar51 arthur do val prefeito 51 #domingoe51 #arrancada51 arthur do val prefeito mamãe falei vai ser prefeito de são paulo vote 51
@roxmo @lzmrtn66 @reclameaqui @loggi vote mamãe falei para prefeit

TRANSLATED: @ nikoh_76 @ peterjordan100 vote mommy told mayor of são paulo 51 # vaidar51 # domingoe51 arthur do val mayor mommy told 51 # arrancada51


@peterjordan100 vote mamãe falei para prefeito de são paulo 51 #vaidar51 #domingoe51 arthur do val prefeito mamãe falei 51 #arrancada51
@peterjordan100 vote mamãe falei para prefeito de são paulo 51 #vaidar51 #domingoe51 arthur do val prefeito mamãe falei 51 #arrancada51
TRANSLATED: @ peterjordan100 vote mommy said to mayor of são paulo 51 # vaidar51 # domingoe51 arthur do val mayor mommy said 51 # arrancada51


@sbtonline @libertadoresbr vote mamãe falei para prefeito de são paulo 51 #vaidar51 #domingoe51 arthur do val prefeito mamãe falei 51 #arrancada51
@sbtonline @libertadoresbr vote mamãe falei para prefeito de são paulo 51 #vaidar51 #domingoe51 arthur do val prefeito mamãe falei 51 #arrancada51
TRANSLATED: @sbtonline @libertadoresbr vote mommy i told mayor of são paulo 51 # vaidar51 # domingoe51 arthur do val mayor mommy said 51 #

TRANSLATED: @ jniorvi33848396 @ meldreams99 I hope that Arthur do Val wins in the first or second round


@rabreder @celsorussomanno russomano não voto, candidato fraco, fui ler o planos e governo dele, me parece ser pior que o do covas, eu voto arthur do val 51
@rabreder @celsorussomanno russomano não voto, candidato fraco, fui ler o planos e governo dele, me parece ser pior que o do covas, eu voto arthur do val 51
TRANSLATED: @rabreder @celsorussomanno russomano not voting, weak candidate, I went to read his plans and government, it seems to me to be worse than that of covas, I vote arthur do val 51


candidato afirma que não devemos nos basear em ideologias e sim em dados e evidências científicas
candidato afirma que não devemos nos basear em ideologias e sim em dados e evidências científicas
TRANSLATED: candidate states that we should not be based on ideologies but on scientific data and evidence


@sbtonline @pfofocalizando vote mamãe falei para prefeito de são paulo 51 #vaidar51 

TRANSLATED: @priscilllando the people of sp have to make an effort to put arthur from val 51 in the second round and take out the pits


@fi_do_ocon dps fala que apoia o arthur do val vc é uma farsa, ogado jr
@fi_do_ocon dps fala que apoia o arthur do val vc é uma farsa, ogado jr
TRANSLATED: @fi_do_ocon dps says you support arthur do val you are a fake, ogado jr


@cesarfilho @jluiztuittecom se eu votasse em sp, arthur do val.
@cesarfilho @jluiztuittecom se eu votasse em sp, arthur do val.
TRANSLATED: @cesarfilho @jluiztuittecom if i voted for sp, arthur do val.


cara os vídeos de campanha do @arthurmoledoval tá foda demais, mano nunca vi um cara tão criativo sem dinheiro público vlw arthur do val vc é fera
cara os vídeos de campanha do @arthurmoledoval tá foda demais, mano nunca vi um cara tão criativo sem dinheiro público vlw arthur do val vc é fera
TRANSLATED: man @ campaign videos from @arthurmoledoval is fucking awesome, man I’ve never seen such a creative guy without public mone

In [134]:

tweets_sentiment = pd.read_csv('tweets/Boulos_translated_sentiment.csv', header=None,  names=['id', 'tweet', 'sentiment', 'tweetTranslated'])

parsed_tweet = {}
tweets = []
for tweet, sentiment in zip(tweets_sentiment['tweet'].values, tweets_sentiment['sentiment'].values):
    parsed_tweet['sentiment'] = sentiment
    parsed_tweet['text'] = tweet
    tweets.append(parsed_tweet)
    parsed_tweet = {}

ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']

tweetsPositives = [tweet['text'] for tweet in ptweets]
tweetsNegatives = [tweet['text'] for tweet in ntweets]
tweets_sentiment.head()
len(tweets_sentiment['tweet'].values)


2319

In [128]:
ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']

In [136]:
print("Total Tweets: ", len(tweets))
print("Postive Tweets: ", len(tweetsPositives))
print("Negative Tweets: ", len(tweetsNegatives))

Total Tweets:  2319
Postive Tweets:  925
Negative Tweets:  587


In [137]:
wordsPositive = count_word(tweetsPositives)
for key, value in wordsPositive:
    print(str(value) + "," + key)

824,boulos
255,turno
182,segundo
158,covas
108,paulo
100,ganhar
97,@guilhermeboulos
82,vencer
79,2°
78,russomano
76,esperança
75,boulos,
73,frança
68,voto
67,eua
67,rumo
66,bons
66,chegando
63,volta
63,ódio
62,democracia
61,ares
61,sobrevoam
61,argentina,
61,bolívia,
61,chile,
61,respirar
58,votar
57,erundina
54,pesquisa
53,primeiro
52,votos
50,esquerda
50,campanha
49,cidade
49,passou
45,pt
42,#virasp50
38,tatto
37,candidato
36,russomanno
36,américa,
35,feliz
34,prefeito
33,pesquisas
32,eleição
32,lugar
31,américa
29,maior
28,chance
28,turno,
28,ibope
26,começaram
25,demais
25,bruno
25,passaram
25,@joelpinheiro85
25,márcio
24,bolsonaro
24,útil
24,sp,
23,ganha
23,50
22,cara
22,eleições
22,psol
22,grande
22,tanto
21,eleger
21,2º
20,@luizaerundina
20,frente
20,direita
19,candidatura
19,chances
19,13%
19,prefeitura
18,amor
18,deus
18,muita
18,arthur
18,hora
18,guilherme
18,deve
17,estar
17,paulo,
16,#bouloserundina50
16,vitória
16,tirar
16,caiu
16,apoiar
16,chegar
16,virar
16,vem
15,tempo


1,caralho,
1,taaaaanta
1,emoção
1,reto,
1,militante
1,encheu
1,vitória,
1,reacendeu
1,servirá
1,chapa,
1,venceremos
1,russoumano
1,ner
1,entregar
1,sujeito
1,cuja
1,depredar
1,alheio
1,chave
1,cofre
1,rica
1,arrojada
1,camarote
1,óbvio,
1,preocupação
1,projecções
1,prova
1,solta
1,bolsonaro,
1,preso
1,tucanos
1,@dricalyz
1,@sgtogetulio
1,proposital
1,confundir
1,deixam
1,considerarmos
1,altamente
1,18%
1,*20%
1,engraçado
1,centímetro
1,miséria
1,arriscando
1,roleta
1,unfollow),
1,existencial
1,profissional
1,bolao
1,deficit
1,transferindo
1,coisinha
1,privada
1,ganhe,
1,esquerdismo
1,funciona
1,risada
1,fica,
1,claramente
1,retirado
1,revelação
1,identidade
1,laurinha
1,dica
1,nunca,
1,forças
1,desenha
1,páreo
1,tectectec
1,sapatinho
1,piso
1,taco
1,cecília
1,@passivo1994
1,perdida
1,desastre
1,cm
1,4️⃣5️⃣
1,moderação,
1,estremos
1,@xicosa
1,@rehcanceriano
1,lavagem
1,cerebral
1,força,
1,@aciollimi
1,(40%
1,desconhecimento),
1,(muito
1,periferias),
1,comunicador
1,(por
1,tv)
1,gostava


In [138]:
wordsNegative = count_word(tweetsNegatives)
for key, value in wordsNegative:
    print(str(value) + "," + key)

550,boulos
258,turno
255,segundo
121,covas
64,@guilhermeboulos
62,frança
58,boulos,
53,russomano
51,russomanno
49,votar
48,paulo
41,pesquisa
39,#virasp50
38,ibope
38,voto
36,esquerda
36,pt
31,turno,
31,2º
29,virar
28,candidato
24,jogo
23,2°
23,erundina
22,pior
21,votos
21,tatto
21,cidade
21,pesquisas
20,bruno
18,frente
18,psol
18,cara
17,levar
17,#bouloserundina50
17,caralho
17,prefeito
16,apoio
16,ruim
15,márcio
15,bolsonaro
15,vota
14,campanha
14,indo
13,@arthurmoledoval
13,50
13,@celsorussomanno
13,covas,
13,contra
12,lugar
12,primeiro
12,chegar
11,prefeitura
11,paulo,
11,@carteiroreaca
11,apoiar
10,amplia
10,eleição
10,sp:
10,disputam
10,vaga
10,qualquer
10,vem
10,candidatura
10,esperança
10,política
10,acredito
10,porra
10,tempo
9,psdb
9,paulistanos
9,invasor
9,passou
9,lula
9,pesquisas,
9,colocar
9,virou
9,maior
9,ganha
9,chances
9,hein
9,pouco
9,guilherme
9,outro
9,fake
9,difícil
9,social
8,cima
8,subiu
8,deus
8,13%
8,diferença
8,merece
8,triste
8,russomano,
8,ódio
8,rejeição
8,

In [147]:
clusters_df = pd.read_csv('tweets_boulos_5_clusters.csv', header=None,  names=['tweet', 'cluster'])

In [148]:
clusters_df.head()

,tweet,cluster
0,vote50capas bons ares sobrevoam a américa co...,0
1,dudadellorusso guilhermeboulos bons ares sob...,0
2,monicaseixas guilhermeboulos bons ares sobre...,0
3,romulo maia98 bons ares sobrevoam a américa ...,0
4,isapennapsol bons ares sobrevoam a américa c...,0


In [149]:
def count_word_in_tweets_by_cluster(texts, clusters, cluster_selected):
    dict_words = {}
    
    index = 0
    for cluster in clusters:
        if(int(cluster) == int(cluster_selected)):
            text = texts[index]
            words = text.split(' ')
            #print(text)
            #print('\n\n')

            for word in words:
                word = word.replace('!', '')
                word = word.replace('?', '')
                word = word.replace('.', '')
                word_is_ok = True 
                if word not in stop_words:
                    if word in dict_words:
                        count = dict_words[word]
                        count += 1
                        dict_words[word] = count 
                    else:
                        dict_words[word] = 1
        index += 1

    dictWordsSorted = sorted(dict_words.items(), key=lambda x:x[1], reverse=True)

    return dictWordsSorted

In [153]:
dictWordsSorted_cluster = count_word_in_tweets_by_cluster(clusters_df['tweet'].tolist(), 
                                                          clusters_df['cluster'].tolist(),
                                                         3)
dictWordsSorted_cluster

[('boulos', 954),
 ('turno', 220),
 ('segundo', 209),
 ('covas', 157),
 ('guilhermeboulos', 92),
 ('frança', 74),
 ('russomano', 73),
 ('ganhar', 54),
 ('pt', 53),
 ('virasp50', 48),
 ('erundina', 47),
 ('pesquisa', 47),
 ('russomanno', 45),
 ('esquerda', 45),
 ('campanha', 44),
 ('pesquisas', 39),
 ('primeiro', 34),
 ('joelpinheiro85', 34),
 ('prefeito', 33),
 ('cara', 33),
 ('tatto', 33),
 ('candidato', 31),
 ('guilherme', 30),
 ('bruno', 30),
 ('psol', 29),
 ('frente', 29),
 ('ibope', 28),
 ('eleição', 27),
 ('cidade', 26),
 ('márcio', 25),
 ('paulo', 24),
 ('vem', 23),
 ('lugar', 23),
 ('lula', 23),
 ('virar', 22),
 ('13', 22),
 ('maior', 21),
 ('chances', 20),
 ('virou', 19),
 ('2º', 19),
 ('bouloserundina50', 18),
 ('bolsonaro', 18),
 ('chegar', 18),
 ('prefeitura', 17),
 ('2°', 17),
 ('arthur', 16),
 ('deus', 16),
 ('ganha', 15),
 ('possível', 15),
 ('falando', 15),
 ('chance', 15),
 ('pior', 15),
 ('eleito', 15),
 ('levar', 15),
 ('grande', 15),
 ('arthurmoledoval', 15),
 ('can